In [39]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
import json

from fastlane_bot import Bot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.data.abi import UNISWAP_V2_POOL_ABI, UNISWAP_V3_POOL_ABI, SUSHISWAP_POOLS_ABI, BANCOR_V3_POOL_COLLECTION_ABI, \
    CARBON_CONTROLLER_ABI
from unittest.mock import Mock
import pytest

print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *

plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.10.1 (07/May/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.1 (2023-07-03)
UniswapV3 v0.0.1 (2023-07-03)
SushiswapV2 v0.0.1 (2023-07-03)
CarbonV1 v0.0.1 (2023-07-03)
BancorV3 v0.0.1 (2023-07-03)
Version = 3-b2.2 [requirements >= 3.0 is met]


/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_72573/1147139390.py:24: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


In [40]:
with open('fastlane_bot/data/event_test_data.json', 'r') as f:
    setup_data = json.load(f)

In [41]:
mocked_contract = Mock()
mocked_contract.functions.token0.return_value.call.return_value = 'token0'
mocked_contract.functions.token1.return_value.call.return_value = 'token1'
mocked_contract.functions.fee.return_value.call.return_value = 3000

In [42]:
uniswap_v2_exchange = UniswapV2()
assert (uniswap_v2_exchange.get_abi() == UNISWAP_V2_POOL_ABI)
assert (uniswap_v2_exchange.get_fee('', mocked_contract) == ('0.003', 0.003))
assert (uniswap_v2_exchange.get_tkn0('', mocked_contract, {}) == mocked_contract.functions.token0().call())

In [43]:
uniswap_v3_exchange = UniswapV3()
assert (uniswap_v3_exchange.get_abi() == UNISWAP_V3_POOL_ABI)
assert (uniswap_v3_exchange.get_fee('', mocked_contract) == (mocked_contract.functions.fee().call(), (float(mocked_contract.functions.fee().call()) / 1000000.0)))
assert (uniswap_v3_exchange.get_tkn0('', mocked_contract, {}) == mocked_contract.functions.token0().call())

In [44]:
sushiswap_v2_exchange = SushiswapV2()
assert (sushiswap_v2_exchange.get_abi() == SUSHISWAP_POOLS_ABI)
assert (sushiswap_v2_exchange.get_fee('', mocked_contract) == ('0.003', 0.003))
assert (sushiswap_v2_exchange.get_tkn0('', mocked_contract, {}) == mocked_contract.functions.token0().call())

In [45]:
bancor_v3_exchange = BancorV3()
assert (bancor_v3_exchange.get_abi() == BANCOR_V3_POOL_COLLECTION_ABI)
assert (bancor_v3_exchange.get_fee('', mocked_contract) == ('0.000', 0.0))
assert (bancor_v3_exchange.get_tkn0('', mocked_contract, setup_data['bancor_v3_event']) == bancor_v3_exchange.BNT_ADDRESS)

In [46]:
carbon_v1_exchange = CarbonV1()
assert (carbon_v1_exchange.get_abi() == CARBON_CONTROLLER_ABI)
assert (carbon_v1_exchange.get_fee('', mocked_contract) == ('0.002', 0.002))
assert (carbon_v1_exchange.get_tkn0('', mocked_contract, setup_data['carbon_v1_event_update']) == setup_data['carbon_v1_event_update']['args']['token0'])

In [47]:
carbon_v1_exchange = CarbonV1()
assert (carbon_v1_exchange.get_abi() == CARBON_CONTROLLER_ABI)
assert (carbon_v1_exchange.get_fee('', mocked_contract) == ('0.002', 0.002))
assert (carbon_v1_exchange.get_tkn0('', mocked_contract, setup_data['carbon_v1_event_create']) == setup_data['carbon_v1_event_create']['args']['token0'])

In [48]:
carbon_v1_exchange = CarbonV1()
assert (carbon_v1_exchange.get_abi() == CARBON_CONTROLLER_ABI)
cid = setup_data['carbon_v1_event_delete']['args']['id']